从`torchvision`载入resnet18预训练模型。

创建一个随机tensor来表示一个带有3个通道、64*64的图像，随机初始化其对应的标签。

In [1]:
import torch,torchvision
model = torchvision.models.resnet18(pretrained=True)
data  = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)

接下来在模型的每一层运行输入数据以做出预测，这是向前传播。

In [2]:
prediction = model(data)

计算loss，然后通过网络反向传播这个loss。

当对这个loss 张量调用`backward()`时，就会启动反向传播。Autograd利用参数的`.grad`属性中计算并存储每个模型参数的梯度。

In [3]:
loss = (prediction - labels).sum()
loss.backward()

载入优化器，learning rate为0.01,momentum为0.9。

In [4]:
optim = torch.optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)
optim

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

最后调用`.step()`启动梯度下降，优化器通过存储在`.grad`中的梯度来调整每个参数。

In [5]:
optim.step()#gradient descent

# autograd的工作原理

用`requires_grad=True`创建两个张量a和b，这个标记表明a和b的每个操作都应该被跟踪。

In [19]:
import torch
a = torch.tensor([2.,3.],requires_grad=True)
b = torch.tensor([6.,4.],requires_grad=True)

In [30]:
x = torch.randn(2,2)
print(x.requires_grad)
x.requires_grad_(True)
print(x.requires_grad)
y = (x * x).sum()
print(y.grad_fn)

False
True


假设a和b是NN的参数，Q是误差。

In [20]:
Q grad_fn3 - b**2

**当我们在Q上调用`.backward()`时，autograd计算这些梯度并将它们存储在各自张量的`.grad`属性中。**

在`Q.backward()`中需要显式传递一个`gradient`参数，因为它是一个向量。`gradient`是一个与Q形状相同的张量，它表示Q自身的梯度，

我们还可以将Q聚合为一个标量并隐式向后调用，如`Q.sum().backward()`。

In [21]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [22]:
print(a.grad)
# 36 = 9*2*2*1
# 810 = 2*3*3*1

tensor([36., 81.])


In [23]:
print(b.grad)
# -12 = -2*6*1
# -80 = -2*4*1

tensor([-12.,  -8.])


In [24]:
# 检查得到的gradient是否正确
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


可以通过将代码包裹在`with torch.no_grad()`，来停止对从跟踪历史中 的`.requires_grad=True`的张量自动求导。

In [32]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


操作的输出张量将需要梯度，即使只有一个输入张量是`requires_grad=True`。

In [34]:
x = torch.rand(5,5)
y = torch.rand(5,5)
z = torch.rand((5,5),requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


在预训练的微调中，冻结大部分模型，通常只修改分类器层以对新标签做出预测。

我们加载一个预先训练好的`resnet18`模型，并冻结所有参数。

In [35]:
from torch import nn,optim
model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

对于resnet，分类器是最后一层的`model.fc`。

我们可以简单地用一个新的线性层(默认未冻结)代替它，作为我们的分类器。

In [36]:
model.fc = nn.Linear(512,10)

现在模型中的所有参数，除了`model.fc`的参数。计算梯度的唯一参数是`model.fc`的权值和偏差。

In [39]:
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(),lr=1e-2,momentum=0.9)

唯一计算梯度(并因此在梯度下降中更新)的参数是分类器的权值和偏差。